In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.multioutput import MultiOutputClassifier
import joblib

In [10]:
df = pd.read_csv('shopping_behavior.csv')

In [11]:

df.head(3)

,Age,Gender,Item Purchased,Category,Purchase Amount (USD),State,Size,Color,Season,Review Rating,Subscription Status,Shipping Type,Discount Applied,Promo Code Used,Previous Purchases,Payment Method,Frequency of Purchases,City,Region
0,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,Louisville,South
1,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,Louisville,South
2,55,Male,Blouse,Clothing,53,Kentucky,L,Gray,Winter,3.1,Yes,Express,Yes,Yes,14,Venmo,Fortnightly,Lexington,South


In [12]:
names = df.columns
types = df.dtypes

scaler = MinMaxScaler()

label_encoders = {}
for col in df.columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Save the label encoders
for col, le in label_encoders.items():
    joblib.dump(le, f'./encoder/label_encoder_{col.lower()}.pkl')


In [14]:
X_trend = df[['Age', 'Gender', 'Category', 'State',  'Season']]
Y_trend = df['Item Purchased']
X_trend_scaled = scaler.fit_transform(X_trend)
model_trend = RandomForestClassifier(random_state=42)
model_trend.fit(X_trend_scaled, Y_trend)


RandomForestClassifier(random_state=42)

In [15]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, LabelEncoder
def predict_trend_item(age, gender, category, state, season):
    new_customer_data = pd.DataFrame({
        'Age': [age],
        'Gender': [gender],
        'Category': [category],
        'State': [state],
        'Season': [season]
    })
    for col in ['Age', 'Gender', 'Category', 'State', 'Season']:
        if new_customer_data[col][0] not in label_encoders[col].classes_:
            print(f"Giá trị '{new_customer_data[col][0]}' không có trong label encoder của cột '{col}'")
            return None
        new_customer_data[col] = label_encoders[col].transform(new_customer_data[col])
    new_customer_data = new_customer_data[['Age', 'Gender', 'Category', 'State', 'Season']]

    new_customer_data_scaled = scaler.transform(new_customer_data)
    predicted_item = model_trend.predict(new_customer_data_scaled)
    predicted_item_name = label_encoders['Item Purchased'].inverse_transform(predicted_item)
    print("Dự đoán sản phẩm cho khách hàng:", predicted_item_name)
    return predicted_item_name


In [16]:
item = predict_trend_item(30, 'Female', 'Accessories', 'Kentucky','Summer')

Dự đoán sản phẩm cho khách hàng: ['Belt']


In [17]:
import joblib

In [18]:
joblib.dump(model_trend, 'model_trend.pkl')

joblib.dump(scaler, './scaler/scaler.pkl')

['./scaler/scaler.pkl']